In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import pprint

from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split

from collections import Counter
%matplotlib inline

In [ ]:
# load toy data
iris = load_iris()
df = pd.DataFrame(iris.data, columns=iris.feature_names)
df['label'] = iris.target
df.columns = ['sepal length', 'sepal width', 'petal length', 'petal width', 'label']
print(df.head())

# X, Y
data = np.array(df.iloc[:100, [0, 1, -1]])
X, Y = data[:,:-1], data[:,-1]
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2)
print(X_train.shape, X_test.shape, Y_train.shape, Y_test.shape, )
print(Y_train, Y_test)

# plot dataset
plt.scatter(X[:50,0], X[:50,1], label='0')
plt.scatter(X[50:100,0], X[50:100,1], label='1')
plt.xlabel('sepal length')
plt.ylabel('sepal width')
plt.legend()

# =====================暴 力 算 法=====================

In [72]:
class KNN_Origin:
    def __init__(self, X_train, Y_train, X_test, Y_test, K=2, p=2):
        self.k = K
        self.p = max(1,p)
        self.X_train = X_train
        self.Y_train = Y_train
        self.X_test = X_test
        self.Y_test = Y_test
                
    def calDistance(self, x, y):
        '''
        input:
            x: exmNum*feaNum
            y:      1*feaNum
            p: type of distance
        output: N*1 distance between x and y
        '''
        dis = np.power(np.sum(np.power(np.abs(x-y),self.p),1),1/self.p)
        return dis
    
    def predict(self, x):
        diss = self.calDistance(self.X_train, x)
        diss_idx = np.argsort(diss)
        top_k_idx = diss_idx[:self.k]
        top_k_diss = diss[top_k_idx]
        top_k_x = self.X_train[top_k_idx]
        top_k_y = self.Y_train[top_k_idx]
        counter = Counter(top_k_y)
        predLabel = counter.most_common()[0][0]
        return predLabel, top_k_x, top_k_diss
    
    def score(self,):
        rightNum = 0
        for x, y in zip(self.X_test, self.Y_test):
            predLabel = self.predict(x)[0]
            if predLabel == y:
                rightNum += 1
        return rightNum / len(self.X_test)

In [ ]:
K = 10
p = 2
knn1 = KNN_Origin(X_train, Y_train, X_test, Y_test, K, p)
acc = knn1.score()
print("Val of K: ",K)
print("Val of p: ",p)
print("Accuracy: ",acc*100,"%")

In [ ]:
unknownX = np.array([5.5,3])
predlabel = int(knn1.predict(unknownX)[0])

# plot dataset
plt.scatter(X[:50,0], X[:50,1], label='0')
plt.scatter(X[50:100,0], X[50:100,1], label='1')
plt.scatter(unknownX[0], unknownX[1], label=predlabel)
plt.xlabel('sepal length')
plt.ylabel('sepal width')
plt.legend()

In [ ]:
val_K = [i for i in range(1,len(X_train)+1)]
acc_K = [KNN_Origin(X_train, Y_train, X_test, Y_test, val_k, 2).score() for val_k in val_K]
plt.plot(val_K, acc_K, marker='D', markersize=2, label="acc")
plt.xlabel("the value of K")
plt.ylabel("the accuracy of test set")
plt.title("the influence of K")
plt.legend()

# =====================最近邻算法-kd树=====================

### 一、构建KD树
（0）坐标轴选取方式：
    1、顺序轮训
    2、特征方差大者优先（方差大意味着该特征能较好区分不同类别）
    
（1）选取坐标轴（特征），找到该坐标轴（特征）上的中位数实例，垂直该轴并通过该实例划分区域

（2）重复

（3）直到两个子区域没有实例存在即构建完成。

### 二、查找KD树
（1）从root节点开始，DFS搜索直到叶子节点，同时在stack中顺序存储已经访问的节点。

（2）如果搜索到叶子节点，当前的叶子节点被设为最近邻节点。

（3）然后通过stack回溯:

    1、如果当前点的距离比最近邻点距离近，更新最近邻节点。
    
    2、然后检查以最近距离为半径的圆是否和父节点的超平面相交：
    
        2.1、如果相交，则必须到父节点的另外一侧，用同样的DFS搜索法，开始检查最近邻节点。
        
        2.2如果不相交，则继续往上回溯，而父节点的另一侧子节点都被淘汰，不再考虑的范围中。
        
（4）当搜索回到root节点时，搜索完成，得到最近邻节点。

###### （代码详见https://github.com/hktxt/Learn-Statistical-Learning-Method）

In [101]:
def distance(x, y, p=2):
    """calculate type of P distance between two points.
    input:
        x: N*M shape array.
        y: 1*M shape array.
        p: type of distance
        
    output:
        N*1 shape of distance between x and y.
    """
    try:
        dis = np.power(np.sum(np.power(np.abs((x - y)), p), 1), 1/p)
    except:
        dis = np.power(np.sum(np.power(np.abs((x - y)), p)), 1/p)
    
    return dis

In [102]:
class KdTree:
    """
    build kdtree recursively along axis, split on median point.
    k:      k dimensions
    method: alternate/variance, 坐标轴轮替或最大方差轴
    """
    
    def __init__(self, k=2, method='alternate'):
        self.k = k
        self.method = method
        
    def build(self, points, depth=0):
        n = len(points)
        if n <= 0:
            return None
        
        if self.method == 'alternate':
            axis = depth % self.k
        elif self.method == 'variance':
            axis = np.argmax(np.var(points, axis=0), axis=0)
        
        sorted_points = sorted(points, key=lambda point: point[axis])
        
        return {
            'point': sorted_points[n // 2],
            'left': self.build(sorted_points[:n//2], depth+1),
            'right': self.build(sorted_points[n//2+1:], depth+1)
        }

In [103]:
data = np.array([[2,3],[5,4],[9,6],[4,7],[8,1],[7,2]])

kd1 = KdTree(k=2, method='alternate')
tree1 = kd1.build(data)

kd2 = KdTree(k=2, method='variance')
tree2 = kd2.build(data)

In [ ]:
pp = pprint.PrettyPrinter(indent=4)
pp.pprint(tree1) # equal to figure. 3.4 《统计学习方法》
pp.pprint(tree2) # 在该数据集上两种方法结果一样

In [105]:
class SearchKdTree:
    """
    search closest point
    """
    def __init__(self, k=2):
        self.k = k
        
    def __closer_distance(self, pivot, p1, p2):
        if p1 is None:
            return p2
        if p2 is None:
            return p1
        
        d1 = distance(pivot, p1)
        d2 = distance(pivot, p2)

        if d1 < d2:
            return p1
        else:
            return p2
    
    def fit(self, root, point, depth=0):
        if root is None:
            return None
        
        axis = depth % self.k
        
        next_branch = None
        opposite_branch = None
        
        if point[axis] < root['point'][axis]:
            next_branch = root['left']
            opposite_branch = root['right']
        else:
            next_branch = root['right']
            opposite_branch = root['left']
            
        best = self.__closer_distance(point,
                                     self.fit(next_branch,
                                             point,
                                             depth+1),
                                     root['point'])
        
        if distance(point, best) > abs(point[axis] - root['point'][axis]):
            best = self.__closer_distance(point,
                                     self.fit(opposite_branch,
                                             point,
                                             depth+1),
                                     best)
            
        return best

In [106]:
# test
point = [3.,4.5]

search = SearchKdTree()
best = search.fit(tree1, point, depth=0)
print(best)

[2 3]
